In [ ]:
import mbuild as mb

import gmso
from gmso import ForceField
from gmso.external import convert_mbuild
from gmso.formats.lammpsdata import write_lammpsdata
from mbuild.formats.lammpsdata import write_lammpsdata

In [ ]:
# Load in single water TIP3P structure

water = mb.load("files/tip3p.mol2")
water = water.children[0]
water.name = "water"

# element_map = which site name corresponds to which atom_type name. 
# In the future atomtyping will be done through foyer. 
element_map = {"O": "opls_111",
               "H": "opls_112"}

In [ ]:
# Fill a box with 1000 water molecule

water_box = mb.fill_box(
    compound=water,     
    n_compounds=1000,
    density=1000,
)

In [ ]:
# Load in topology forcefield

forcefield = ForceField("files/tip3p.xml")

In [ ]:
# Generate a topology from the mbuild compound

top = convert_mbuild.from_mbuild(water_box)

In [ ]:
# Assign atom types
for atom in top.sites:
    atom.atom_type = forcefield.atom_types[element_map[atom.name]]
    
# Assign bond types
for bond in top.bonds:
    bond.bond_type = bond.connection_type = forcefield.bond_types["opls_111~opls_112"]

# Create angles with correct atom type and add to top
for subtop in top.subtops:
    angle = gmso.core.angle.Angle(
        connection_members=[site for site in subtop.sites],
        name="opls_112~opls_111~opls_112",
        connection_type=forcefield.angle_types["opls_112~opls_111~opls_112"]
    )
    top.add_connection(angle, update_types=False)

top.update_topology()

In [ ]:
# Write out LAMMPS Files
write_lammpsdata(top, 'data.tip3p')

In [ ]:
# Run the lammps simulation
!lmp_serial < files/in.tip3p